In [35]:
import base64
import drawSvg as draw
# import drawsvg as drawp
from IPython.display import display, HTML
from io import StringIO

blue = "rgb(83,160,227)"
trans = "none"

class Icon:
    def __init__(self, width, height, stroke, stroke_width):
        self.width = width
        self.height = height
        self.stroke = stroke
        self.stroke_width = stroke_width
        self.d = draw.Drawing(width, height)
        
    def add_rect(self, x, y, w, h, fill="none", stroke=None, stroke_width=None, opacity=1.0):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Rectangle(x, y, w, h, fill=fill, stroke=stroke, stroke_width=stroke_width, stroke_linejoin="round", fill_opacity=opacity)
        )
        
    def add_lines(self, *points, fill="none", close=True, stroke=None, stroke_width=None, stroke_linejoin="round", stroke_linecap="round", opacity=1.0):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Lines(*points, close=close, fill=fill, stroke=stroke, stroke_width=stroke_width, stroke_linejoin=stroke_linejoin, stroke_linecap=stroke_linecap, fill_opacity=opacity)
        )
        
    def add_arc(self, cx, cy, r, startDeg, endDeg, fill="none", stroke=None, stroke_width=None):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Arc(cx, cy, r, startDeg, endDeg, cw=False, stroke=stroke, stroke_width=stroke_width, fill=fill, stroke_linecap="round")
        )

    def add_ellipse(self, cx, cy, rx, ry, fill="none", stroke=None, stroke_width=None):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Ellipse(cx, cy, rx, ry, fill=fill, stroke=stroke, stroke_width=stroke_width)
        )

    def add_text(self, text, fontsize, x, y, font_family="sans-serif", font_weight="normal", fill="none"):
        self.d.append(
            draw.Text(text, fontsize, x, y, fill=fill, font_family=font_family, font_weight=font_weight)
        )
        
    def as_svg(self, name, width, height):
        outputFile = StringIO()
        outputFile.write('''<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{}" height="{}" viewBox="{} {} {} {}">'''.format(
                width, height, *self.d.viewBox))
        outputFile.write('\n')

        isDuplicate = lambda x: False

        for element in self.d.allElements():
            element.writeSvgElement(name, isDuplicate, outputFile, False)
            outputFile.write('\n')
        outputFile.write('</svg>\n')

        return outputFile.getvalue()


class Icons:
    def __init__(self, width, height, background):
        self.width = width
        self.height = height
        self.background = background
        self.icons = {}
        
    def append(self, name, icon):
        self.icons[name] = icon
        
    def show(self, theme, kind, show_html=False):
        html = f"""<style>
    .btn_{kind}_{theme} {{
        display: inline-block;
        width: {self.width}px;
        height: {self.height}px;
        background-color: {self.background}; 
        padding: 0px;
        margin: 4px;
        border: solid 1px #222;
    }}
    .all_{theme} {{
        background-color: {self.background};     
    }}
    </style>"""

        for name, icon in self.icons.items():
            html += f'<div class="btn_{kind}_{theme}">{icon.as_svg(name, self.width, self.height)}</div>\n'
        if show_html:
            print(html)
        else:
            display(HTML(f"""<div class="all_{theme}">{html}</div>"""))

    def save(self, theme):
        for name, icon in self.icons.items():
            xml = icon.as_svg(name, self.width, self.height)
            with open(f"./icons/{theme}/{name}.svg", "w") as fd:
                fd.write(xml)

In [36]:
def cube(name, width, height, margin, stroke, stroke_width,
         ftop="none", fbottom="none", ffront="none", frear="none", fright="none", fleft="none"):
    m = margin
    r = 2 * (width - m) / 3   # rect size
    o = r / 2                 # offset
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(o, o, r,  r, fill=frear)
    icon.add_lines(m,m, m,m+r, o,o+r, o,o, fill=fleft)
    icon.add_lines(m,m, o,o, o+r,o, r+m,m, fill=fbottom)
    icon.add_lines(m,m+r, o,o+r, o+r,o+r, r+m,m+r, fill=ftop)
    icon.add_lines(m+r,m, m+r,m+r, o+r,o+r, o+r,o, fill=fright)
    icon.add_rect(m, m, r, r, fill=ffront)
    return icon

def iso(width, height, margin, stroke, stroke_width):
    x2 = 2 * margin
    x1 = width / 2
    x0 = width - 2 * margin
    y4 = margin
    y3 = margin + (width - 2 * margin) / 4
    y2 = margin + (width - 2 * margin) / 2
    y1 = margin + (width - 2 * margin) / 4 * 3
    y0 = width - margin
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(x0,y3, x0,y1, x1,y2, x1,y4, fill=blue)
    icon.add_lines(x1,y2, x1,y4, x2,y3, x2,y1, fill=blue)
    icon.add_lines(x0,y1, x1,y0, x2,y1, x1,y2, fill=blue)
    return icon
   
def reset(width, height, margin, stroke, stroke_width):
    stroke_width *= 1.25
    cx = cy = width / 2
    r1 = width / 2  - 1.5 * margin
    r2 = r1 - 2 * margin
    r = r1 - margin
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_arc(cx, cy, r1, 60, 360)
    icon.add_arc(cx, cy, r2, 60, 360)
    icon.add_lines(
        cx + r2 - margin, cy, 
        cx + r2 + margin, cy + 2 * margin,
        cx + r1 + margin, cy,
        close=False
    )
    return icon

def resize(width, height, margin, stroke, stroke_width):
    stroke_width *= 1.25
    m = margin
    r = width - 2 * m
    d = 2 * m
    md = 3 * m
    mo = 5 * m
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(m, m, r, r)
    icon.add_lines(md, md, mo, mo)
    icon.add_lines(width - md, height - md, width - mo, height - mo)
    icon.add_lines(width - md, md, width - mo, mo)
    icon.add_lines(md, height - md, mo, height - mo)
    icon.add_lines(md, md, md, md + d)
    icon.add_lines(md, md, md + d, md)
    icon.add_lines(width - md, height - md, width - md, height - md - d)
    icon.add_lines(width - md, height - md, width - md - d, height - md)
    icon.add_lines(md, height - md, md, height - md - d)
    icon.add_lines(md, height - md, md + d, height-md)
    icon.add_lines(width - md, md, width-md, md + d)
    icon.add_lines(width - md, md, width - md - d, md)
    return icon

def plane(width, height, margin, stroke, stroke_width):
    stroke_width *= 1.25
    m = margin
    x0 = (margin, 2.5 * margin)
    x1 = (2.2 * margin, width / 2)
    x2 = (width - margin, width / 2)
    x3 = (width - 2.5 * margin, 2.5 * margin)
    c = width / 2
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(*x0, *x1, *x2, *x3)
    icon.add_lines(c,  1.5 * m + height/4, c, c + height/4, stroke=blue, stroke_width=1.5*stroke_width)
    icon.add_lines(c - m, c + height/4 - m, c, c + height/4, c + m, c + height/4 - m, stroke=blue, stroke_width=1.5*stroke_width)
    
    return icon

def nav_icons(factor, stroke, background, missing=None):
    stroke_width = factor / 8 * 1.2
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)

    for name in ["top", "bottom", "front", "rear", "left", "right"]:
        icon = cube(name, width, height, margin, stroke, stroke_width, **{f"f{name}": blue})
        icons.append(name, icon)

    icons.append("iso", iso(width, height, margin, stroke, stroke_width))
    icons.append("reset", reset(width, height, margin, stroke, stroke_width))
    icons.append("resize", resize(width, height, margin, stroke, stroke_width))
    icons.append("plane", plane(width, height, margin, stroke, stroke_width))

    return icons

In [37]:
def mesh(width, height, margin, stroke, stroke_width, mix=False, empty=False, no=False):
    m = margin
    dx = (width - 4 * margin) / 2
    dy = (height - 2 * margin) / 3
    d =  m
    x0 = margin
    x1 = margin + dx
    x2 = margin + 2 * dx
    y0 = margin
    y1 = margin + dy
    y2 = margin + 2 * dy
    y3 = margin + 3 * dy
    o = 2 * margin
    
    if empty:
        stroke = blue
    if no:
        fill = stroke
    else:
        fill = blue
    icon = Icon(width, height, stroke, stroke_width)
    if not mix:
        icon.add_lines(
            x0 + o, y0, x0, y1, x0, y2, x0 + o, y3,
            x1 + o, y3, x1, y2, x1, y1, x1 + o, y0,
            fill=fill, stroke=stroke, stroke_linecap="square", stroke_linejoin="round"
        )
        icon.add_lines(
            x1 + o, y0, x1, y1, x1, y2, x1 + o, y3,
            x2 + o, y3, x2, y2, x2, y1, x2 + o, y0,
            fill=fill, stroke=stroke, stroke_linecap="square", stroke_linejoin="round"
        )
        if not empty:
            icon.add_lines(
                x0, y1, x0, y2, x1, y2, x2, y2, x2, y1, stroke_linecap="square", stroke_linejoin="round"
            )
    else:
        icon.add_lines(
            x0 + o, y3, x1 + o - d, y3, x1 - d, y2, x1 - d, y1, x1 + o - d, y0, 
            x0 + o, y0, x0, y1, x0, y2, stroke=blue, fill=blue, stroke_linejoin="square"
        )            
        icon.add_lines(
            x1 + o + d, y3, x2 + o, y3, x2, y2, x2, y1, x2 + o, y0, 
            x1 + o + d, y0, x1 + d, y1, x1 + d, y2, stroke=blue, fill=blue, stroke_linejoin="round"
        )            
        icon.add_lines(
            x1, y2, x0, y2, x0, y1, x1, y1, close=False
        )
        icon.add_lines(
            x1 + o, y3, x0 + o, y3, x0, y2, x0, y1, x0 + o, y0, x1 + o, y0, close=False
        )
           
    return icon

def shape(width, height, margin, stroke, stroke_width, mix=False, empty=False, no=False):
    m = margin
    cx = width / 2
    cy = height / 2
    r = (width - 2*m) / 2
    r2 = width / 5
    r3 = width / 12
    
    e1 = 30
    e2 = 180 - e1
    e3 = 90 if mix else e1

    icon = Icon(width, height, stroke, stroke_width)
    if not empty:
        if mix:
            icon.add_arc(cx, cy, r2, 90, 270, fill=blue, stroke=trans, stroke_width=stroke_width)
            icon.add_arc(cx, cy, r3, 90, 270, fill=stroke)    
            icon.add_arc(cx, cy-2.5*m, r, e1, e2/2-m, stroke_width=stroke_width)
            icon.add_arc(cx, cy+2.5*m, r, m-e2/2, -e1, stroke_width=stroke_width)
        else:
            icon.add_ellipse(cx, cy, r2, r2, fill=stroke if no else blue, stroke_width=stroke_width, stroke=trans)
            if not no:
                icon.add_ellipse(cx, cy, r3, r3, fill=stroke)    
    icon.add_arc(cx, cy-2.5*m, r, e3, e2, fill=trans, stroke_width=stroke_width)
    icon.add_arc(cx, cy+2.5*m, r, -e2, -e3, fill=trans, stroke_width=stroke_width)
    return icon

def tree_icons(factor, stroke, background, missing):
    stroke_width = factor / 8 * 1.25
    margin = factor / 4
    width = (factor / 8 * 26 + margin) * 0.9
    height = (factor / 8 * 16 + margin) * 0.9
    icons = Icons(width, height, background)

    icons.append("shape", shape(width, height, margin, stroke, stroke_width))
    icons.append("shape_mix", shape(width, height, margin,stroke, stroke_width, mix=True))
    icons.append("shape_no", shape(width, height, margin,stroke, stroke_width, empty=True))
    icons.append("shape_empty", shape(width, height, margin, missing, stroke_width, no=True))

    icons.append("mesh", mesh(width, height, margin, stroke, stroke_width))
    icons.append("mesh_mix", mesh(width, height, margin, stroke, stroke_width, mix=True))
    icons.append("mesh_no", mesh(width, height, margin, stroke, stroke_width, empty=True))
    icons.append("mesh_empty", mesh(width, height, margin, missing, stroke_width, no=True))

    return icons

In [38]:
def stop(width, height, margin, stroke, stroke_width):
    m = 2 * margin

    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(m, m, width - 2*m, width - 2*m, fill=blue)
    return icon

def pause(width, height, margin, stroke, stroke_width):
    m = 2 * margin

    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(1.3*m, m, 1.4*m, width - 2*m, fill=blue)
    icon.add_rect(width - 2.7*m, m, 1.4*m, width - 2*m, fill=blue)
    return icon
    
def play(width, height, margin, stroke, stroke_width):
    m = 2 * margin
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(1.25*m, m, 1.25*m, height - m, width - 1.25*m, width/2, fill=blue)
    
    return icon

def ani_icons(factor, stroke, background, missing=None):
    stroke_width = factor / 8 * 1.5
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)
    
    icons.append("play", play(width, height, margin, stroke, stroke_width))
    icons.append("pause", pause(width, height, margin, stroke, stroke_width))
    icons.append("stop", stop(width, height, margin, stroke, stroke_width))
    
    return icons

In [39]:
def pin(width, height, margin, stroke, stroke_width):
    m = margin
    dx = width / 2
    dy = height - 3.5*m
    r = 2 * m
    
    d =  m
    stroke_width *= 1.25
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(dx-3*m/2, 6*m, 3* m, 5.5*m,fill=blue)
    icon.add_lines(dx, 1.5*m, dx, 5*m, stroke_width=stroke_width*1.8)
    icon.add_arc(dx, dy, 2.5*m, 30, 150, fill=blue, stroke_width=stroke_width)   
    icon.add_arc(dx, 2.6*m, 5*m, 30, 150, fill=blue, stroke_width=stroke_width)
    icon.add_lines(2.25*m, 5.1*m, width-2.25*m, 5.1*m, stroke_width=stroke_width)
    icon.add_lines(4.4*m, dy+1.2*m, width-4.4*m, dy+1.2*m, stroke_width=stroke_width)   
    return icon

def help(width, height, margin, stroke, stroke_width):
    stroke_width *= 1.25
    cx = cy = width / 2
    r1 = width / 2  - 1.5 * margin
    r2 = r1 - 2 * margin
    r = r1 - margin
    
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_arc(cx, cy, r1, 0, 359)

    icon.add_text("?", margin*10, cx-3*margin, 3*margin, fill=blue, font_weight="bold")
    return icon 

def pin_icons(factor, stroke, background, missing=None):
    stroke_width = factor / 8 
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)

    icons.append("pin", pin(width, height, margin, stroke, stroke_width))
    icons.append("help", help(width, height, margin, stroke, stroke_width))
    
    return icons

theme = "light"
stroke = "#333"
background = "#fff"
missing = "#ddd"
icons = pin_icons(32, stroke, background, missing )
icons.show(theme, "pin")

In [95]:
def measure(width, height, margin, stroke, stroke_width):
    m = margin
    d = 2
    d2 = 3.4
    s1 = stroke_width * 1.5
    s2 = stroke_width * 1.7
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(m, 2.5 * m, m, 10.5 * m, stroke_width=s1)
    icon.add_lines(width - m, 2.5 * m, width - m, 10.5 * m, stroke_width=s1)

    icon.add_lines(d * m, 6.5 * m, width - d * m, 6.5 * m, stroke_width=s2, stroke=blue)

    icon.add_lines(d * m, 6.5 * m, d2 * m, 8.0 * m, stroke_width=s2, stroke=blue)
    icon.add_lines(d * m, 6.5 * m, d2 * m, 5.0 * m, stroke_width=s2, stroke=blue)

    icon.add_lines(
        width - d * m, 6.5 * m, width - d2 * m, 8.0 * m, stroke_width=s2, stroke=blue
    )
    icon.add_lines(
        width - d * m, 6.5 * m, width - d2 * m, 5.0 * m, stroke_width=s2, stroke=blue
    )
    return icon


def size(width, height, margin, stroke, stroke_width):
    m = margin
    r = width - 2 * m
    d = 2
    d2 = 3.1
    s1 = stroke_width * 1.45
    s2 = stroke_width * 1.7

    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(m, m, r, r, stroke=stroke, stroke_width=s1)

    icon.add_lines(d * m, 6.5 * m, width - d * m, 6.5 * m, stroke_width=s2, stroke=blue)
    icon.add_lines(6.5 * m, d * m, 6.5 * m, width - d * m, stroke_width=s2, stroke=blue)

    icon.add_lines(d * m, 6.5 * m, d2 * m, 8.0 * m, stroke_width=s2, stroke=blue)
    icon.add_lines(d * m, 6.5 * m, d2 * m, 5.0 * m, stroke_width=s2, stroke=blue)
    icon.add_lines(
        width - d * m, 6.5 * m, width - d2 * m, 8.0 * m, stroke_width=s2, stroke=blue
    )
    icon.add_lines(
        width - d * m, 6.5 * m, width - d2 * m, 5.0 * m, stroke_width=s2, stroke=blue
    )

    icon.add_lines(6.5 * m, d * m, 8.0 * m, d2 * m, stroke_width=s2, stroke=blue)
    icon.add_lines(6.5 * m, d * m, 5.0 * m, d2 * m, stroke_width=s2, stroke=blue)
    icon.add_lines(
        6.5 * m, width - d * m, 8.0 * m, width - d2 * m, stroke_width=s2, stroke=blue
    )
    icon.add_lines(
        6.5 * m, width - d * m, 5.0 * m, width - d2 * m, stroke_width=s2, stroke=blue
    )

    return icon


from math import cos, radians, sin, acos, asin


def rotate(xorigin, yorigin, x, y, angle):
    x1 = (
        xorigin
        + (x - xorigin) * cos(radians(angle))
        - (y - yorigin) * sin(radians(angle))
    )
    y1 = (
        yorigin
        + (x - xorigin) * sin(radians(angle))
        + (y - yorigin) * cos(radians(angle))
    )
    return x1, y1


def angle2(width, height, margin, stroke, stroke_width):
    m = margin * 2
    xcenter = width / 2
    ycenter = 1.5 * m
    alpha = 45
    f = 0.8  # lower factor of the arc
    r = 0.6 * width 

    x1 = -cos(radians(alpha)) * r + xcenter
    y1 = sin(radians(alpha)) * r + ycenter
    x2 = xcenter
    y2 = ycenter
    x3 = cos(radians(alpha)) * r + xcenter
    y3 = y1

    s1 = stroke_width * 1.45
    s2 = stroke_width * 1.7

    arrow_len = width * 0.15
    arrow_angle = 30

    #left side
    xa2 = -cos(radians(alpha)) * r * f + xcenter
    ya2 = sin(radians(alpha)) * r * f + ycenter
    xa1 = xa2 + arrow_len
    ya1 = ya2
    xa3 = xa1
    ya3 = ya1
    xa1, ya1 = rotate(xa2, ya2, xa1, ya1, alpha + arrow_angle)
    xa3, ya3 = rotate(xa2, ya2, xa3, ya3, alpha-1.5*arrow_angle)
    arrow_pts1 = [xa1, ya1, xa2, ya2, xa3, ya3]

    #right side
    xa2 = cos(radians(alpha)) * r * f + xcenter
    ya2 = sin(radians(alpha)) * r * f + ycenter
    xa1 = xa2 - arrow_len
    ya1 = ya2
    xa3 = xa1
    ya3 = ya1
    xa1, ya1 = rotate(xa2, ya2, xa1, ya1, -alpha - arrow_angle)
    xa3, ya3 = rotate(xa2, ya2, xa3, ya3, -alpha+1.5*arrow_angle)

    arrow_pts2 = [xa1, ya1, xa2, ya2, xa3, ya3]

    pts = [x1, y1, x2, y2, x3, y3]
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_arc(
        xcenter,
        ycenter,
        f * r,
        alpha,
        180 - alpha,
        stroke_width=s2,
        stroke=blue,
    )

    icon.add_lines(*arrow_pts1, close=False, stroke_width=s1, stroke=blue)
    icon.add_lines(*arrow_pts2, close=False, stroke_width=s1, stroke=blue)
    icon.add_lines(*pts, close=False, stroke_width=s1)

    return icon

def properties(width, height, margin, stroke, stroke_width):
    m = margin
    d = 2 * m

    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(d, width-1.1*d, width-d, width-1.1*d, stroke_width=2*stroke_width)
    
    for i in range(2,6):
        o = 0.3*d
        h = width-i*d-o
        icon.add_lines(2.3*d, h, width-d-o, h, stroke=blue, stroke_width=stroke_width*1.4)
        icon.add_ellipse(1.2 * d+o, h, 0.8*stroke_width, 0.8*stroke_width, stroke=blue, fill=blue)
    return icon

def angle(width, height, margin, stroke, stroke_width):
    m = margin
    d = 1.2 * m
    r = 0.6 * width
   
    a = 21
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(d, 1.6*d, width-d, 1.6*d, stroke_width=1.45*stroke_width)
    icon.add_lines(d, 1.6*d, width-5*d, width-1.3*d, stroke_width=1.45*stroke_width)
    icon.add_arc(d, 1.6*d, r, 9, 51, stroke=blue, stroke_width=2*stroke_width)
#    c1, c2 =  m + r * cos(radians(a)), width/2 - (r * sin(radians(a)))
#    icon.add_lines(c1, c2, c1-0.6*m, c2+1.3*m, c1, c2, c1+1.1*m, c2+0.7*m, stroke=blue, stroke_width=1.7*stroke_width)
#    c1, c2 =  m + r * cos(radians(a)), width/2 + (r * sin(radians(a)))    
#    icon.add_lines(c1, c2, c1-0.6*m, c2-1.3*m, c1, c2, c1+1.1*m, c2-0.7*m, stroke=blue, stroke_width=1.7*stroke_width)
    return icon

#def select(width, height, margin, stroke, stroke_width,
#         ftop="none", fbottom="none", ffront="none", frear="none", fright="none", fleft="none"):
#    m = margin*1.25
#    r = 2 * (width - m) / 3   # rect size
#    o = r / 2                 # offset
#    d = 2
#    dd = 0.8
#    icon = Icon(width, height, stroke, stroke_width)
#    icon.add_rect(o, o, r,  r, fill=frear, stroke=stroke, stroke_width=stroke_width)
#    icon.add_lines(m,m, m,m+r, o,o+r, o,o, fill="none", stroke=stroke, stroke_width=dd * stroke_width)
#    icon.add_lines(m,m, o,o, o+r,o, r+m,m, fill="none", stroke=stroke, stroke_width=dd * stroke_width)
#    icon.add_lines(m+r,m, m+r,m+r, o+r,o+r, o+r,o, fill="none", stroke=stroke, stroke_width=stroke_width)
#    icon.add_rect(m, m, r, r, fill="none", stroke=stroke, stroke_width=stroke_width)
#    icon.add_lines(m+r,m+r, o+r,o+r, fill="none", stroke=blue, stroke_width=d * stroke_width)
#    icon.add_lines(m,m, m,m+r, fill="none", stroke=blue, stroke_width=d*stroke_width)
#    icon.add_lines(m,m+r, m+r,m+r, fill="none", stroke=blue, stroke_width=d*stroke_width)
#    icon.add_lines(o+r,o, o+r,o+r, fill="none", stroke=blue, stroke_width=d * stroke_width)
#    return icon

def select(width, height, margin, stroke, stroke_width,
         ftop="none", fbottom="none", ffront="none", frear="none", fright="none", fleft="none"):
    m = margin
    d = 1.2 * m
    r = 0.6 * width
    o = r/2
    dd = 2
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(d, 1.6*d, width-d, 1.6*d, stroke_width=1.45*stroke_width)
    icon.add_lines(d, 1.6*d, o, o+r, o+r, o+r, close=False, fill="none", stroke=blue, stroke_width=1.2*dd * stroke_width)
    icon.add_lines(2.7*d,8*d, o+d,r-2*d,o+3*d,r,close=True, fill="none", stroke=stroke, stroke_width=0.9*dd * stroke_width)
    icon.add_lines(o+2.1*d,r-1.15*d, m+r, m+2.6*d, stroke_width=dd*stroke_width)
    return icon
    
def measure_icons(factor, stroke, background, missing=None):
    stroke_width = factor / 8
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)

    icons.append("measure", measure(width, height, margin, stroke, stroke_width))
    icons.append("properties", properties(width, height, margin, stroke, stroke_width))
    icons.append("angle", angle(width, height, margin, stroke, stroke_width))
    icons.append("select", select(width, height, margin, stroke, stroke_width))
    #icons.append("angle2", angle2(width, height, margin, stroke, stroke_width))

    return icons

theme = "light"
stroke = "#333"
background = "#fff"
missing = "#ddd"
blue = "rgb(48, 142, 225)"
icons = measure_icons(32, stroke, background, missing)
icons.show(theme, "measure")

In [96]:
def transparent(width, height, margin, stroke, stroke_width,
         ftop="none", fbottom="none", ffront="none", frear="none", fright="none", fleft="none"):
    m = margin
    r = 2 * (width - m) / 3   # rect size
    o = r / 2                 # offset
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(o, o, r,  r, fill=frear)
    icon.add_lines(m,m, m,m+r, o,o+r, o,o, fill=fleft)
    icon.add_lines(m,m, o,o, o+r,o, r+m,m, fill=fbottom)
    icon.add_lines(m,m+r, o,o+r, o+r,o+r, r+m,m+r, fill=blue, opacity=0.75)
    icon.add_lines(m+r,m, m+r,m+r, o+r,o+r, o+r,o, fill=blue, opacity=0.75)
    icon.add_rect(m, m, r, r, fill=blue, opacity=0.75)
    return icon

def black_edges(width, height, margin, stroke, stroke_width,
         ftop="none", fbottom="none", ffront="none", frear="none", fright="none", fleft="none"):
    m = margin*1.25
    r = 2 * (width - m) / 3   # rect size
    o = r / 2                 # offset
    d = 1.25
    stroke_width=stroke_width*1.25
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(o, o, r,  r, fill=frear, stroke=blue, stroke_width=d*stroke_width)
    icon.add_lines(m,m, m,m+r, o,o+r, o,o, fill="none", stroke=blue, stroke_width=d*stroke_width)
    icon.add_lines(m,m, o,o, o+r,o, r+m,m, fill="none", stroke=blue, stroke_width=d*stroke_width)
    icon.add_lines(m,m+r, o,o+r, o+r,o+r, r+m,m+r, fill="none", stroke=blue, stroke_width=d*stroke_width)
    icon.add_lines(m+r,m, m+r,m+r, o+r,o+r, o+r,o, fill="none", stroke=blue, stroke_width=d*stroke_width)
    icon.add_rect(m, m, r, r, fill="none", stroke=blue, stroke_width=d*stroke_width)
    return icon

def appearance_icons(factor, stroke, background, missing=None):
    stroke_width = factor / 8 
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)

    icons.append("transparent", transparent(width, height, margin, stroke, stroke_width))
    icons.append("black_edges", black_edges(width, height, margin, stroke, stroke_width))
    
    return icons

theme = "light"
stroke = "#333"
background = "#fff"
missing = "#ddd"
blue = "rgb(48, 142, 225)"
icons = appearance_icons(32, stroke, background, missing )
icons.show(theme, "measure")

In [97]:
from math import sin, cos, radians

def explode(width, height, margin, stroke, stroke_width,
         ftop="none", fbottom="none", ffront="none", frear="none", fright="none", fleft="none"):
    m = margin
    o = 5.5*m
    l = width/2 - o
    r = 4.75*m

    icon = Icon(width, height, stroke, stroke_width)
    for a in range(0,360,72):
        x = (width)/2 + r*cos(radians(a))
        y = (width)/2 + r*sin(radians(a))   

        icon.add_ellipse(x,y,1.25*l,1.25*l, stroke=blue, fill=blue, stroke_width=stroke_width)

        x = (width)/2 + (r-l)*cos(radians(a))
        y = (width)/2 + (r-l)*sin(radians(a))   

        icon.add_lines(width/2, width/2, x, y, stroke=blue, stroke_width=stroke_width/0.8)

    icon.add_ellipse(width/2, width/2,l,l, fill=stroke, stroke_width=stroke_width*1.75)

    return icon

def explode_icons(factor, stroke, background, missing=None):
    stroke_width = factor / 8 
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)

    icons.append("explode", explode(width, height, margin, stroke, stroke_width))
    
    return icons


theme = "light"
stroke = "#333"
background = "#fff"
missing = "#ddd"
blue = "rgb(48, 142, 225)"
icons = explode_icons(32, stroke, background, missing )
icons.show(theme, "explode")

In [98]:
import numpy as np

def axes(width, height, margin, stroke, stroke_width):
    m = margin
    w = stroke_width*2
    d = 3.3*m
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(d,5* m, width-d, width/2-5*m, stroke=xcolor, stroke_width=w)
    icon.add_lines(d,5* m, width-d, width/2+2*m, stroke=ycolor, stroke_width=w)
    icon.add_lines(d,5* m, d, height-m, stroke=zcolor, stroke_width=w)
    
    return icon

def axes0(width, height, margin, stroke, stroke_width):
    m = margin
    w = stroke_width*2
    d = 3.3*m
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(d,5* m, width-d, width/2-5*m, stroke=xcolor, stroke_width=w)
    icon.add_lines(d,5* m, width-d, width/2+2*m, stroke=ycolor, stroke_width=w)
    icon.add_lines(d,5* m, d, height-m, stroke=zcolor, stroke_width=w)

    icon.add_ellipse(d,5* m, m+m/2,1.4*m+m/2,stroke=background, stroke_width=w*1.2)
    icon.add_ellipse(d,5* m, m,1.5*m,fill=background,stroke_width=stroke_width*1.4)
    
    return icon

def grid(width, height, margin, stroke, stroke_width):
    m = margin
    w = stroke_width*2
    d = 1.2*m
    
    icon = Icon(width, height, stroke, stroke_width)
    for x in np.linspace(m, width-m, 4):
        icon.add_lines(d, x, width-d, x, stroke_width=stroke_width*1.5)
    for y in np.linspace(m, width-m, 4):
        icon.add_lines(y, d, y, width-d, stroke_width=stroke_width*1.5)
    
    return icon

def perspective(width, height, margin, stroke, stroke_width):
    m = margin
    w = stroke_width*2
    d = 1.2*m
    w = stroke_width*1.5
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(width-6*m, width-6*m, 5*m, 5*m, stroke=blue, stroke_width=w)
    icon.add_lines(m,m, width-6*m, width-6*m, stroke=blue, stroke_width=stroke_width*0.75)
    icon.add_lines(9*m, 9*m, width-m,width-m, stroke=blue, stroke_width=stroke_width*0.75)
    icon.add_lines(9*m, m, width-m,width-6*m, stroke=blue, stroke_width=stroke_width*0.75)
    icon.add_lines(m, 9*m, width-6*m,width-m, stroke=blue, stroke_width=stroke_width*0.75)
    icon.add_rect(m, m, 8*m, 8*m, stroke=stroke, stroke_width=w*1.25)
    
    return icon

def axes_icons(factor, stroke, background, missing=None):
    stroke_width = factor / 8 
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)

    icons.append("axes", axes(width, height, margin, stroke, stroke_width))
    icons.append("axes0", axes0(width, height, margin, stroke, stroke_width))
    icons.append("grid", grid(width, height, margin, stroke, stroke_width))
    icons.append("perspective", perspective(width, height, margin, stroke, stroke_width))    

    
    return icons

xcolor = "#f00";
ycolor = "rgb(0,128,0)";
zcolor = "#00f";
theme = "light"
stroke = "#333"
background = "#fff"
missing = "#ddd"
blue = "rgb(48, 142, 225)"
icons = axes_icons(32, stroke, background, missing )
icons.show(theme, "axes")

## Show icons

In [99]:
factor = 8
for theme in ["light", "dark"]:

    if theme == "light":
        blue = "rgb(48, 142, 225)"
        stroke = "#333"
        background = "#fff"
        missing = "#ddd"
        xcolor = "#f00";
        ycolor = "rgb(0,128,0)";
        zcolor = "#00f";
    else:
        blue = "rgb(83,160,227)"
        stroke = "#ddd"
        background = "#444"
        missing = "#666"
        xcolor = "rgb(255, 69, 0)";
        ycolor = "rgb(50, 205, 50)";
        zcolor = "rgb(59, 158, 255)";        
        
    icons = nav_icons(factor, stroke, background)
    icons.show(theme, "nav")

    icons = ani_icons(factor, stroke, background)
    icons.show(theme, "ani")
    
    icons = tree_icons(factor, stroke, background, missing)
    icons.show(theme, "tree")
    
    icons = pin_icons(factor, stroke, background, missing)
    icons.show(theme, "pin")
    
    icons = measure_icons(factor, stroke, background, missing)
    icons.show(theme, "measure") 
    
    icons = appearance_icons(factor, stroke, background, missing)
    icons.show(theme, "appearance")  
    
    icons = explode_icons(factor, stroke, background, missing)
    icons.show(theme, "explode")      
    
    
    icons = axes_icons(factor, stroke, background, missing)
    icons.show(theme, "axes")      

## Install icons

In [100]:
import os
import base64

template = """.btn_{mode}_{name} {{
  background:
    url(data:image/svg+xml;base64,{b64})
    no-repeat
    center center;
}}
"""

for theme in  ["light", "dark"]:
    if theme == "light":
        blue = "rgb(83,160,227)"
        stroke = "#444"
        background = "#fff"
        missing = "#ddd"
        xcolor = "#f00";
        ycolor = "rgb(0,128,0)";
        zcolor = "#00f";        
    else:
        blue = "rgb(48, 142, 225)"
        stroke = "#ddd"
        background = "#444"
        missing = "#666"
        xcolor = "rgb(255, 69, 0)";
        ycolor = "rgb(50, 205, 50)";
        zcolor = "rgb(59, 158, 255)";        

    for icons in [
        nav_icons(factor, stroke, background, missing), 
        tree_icons(factor, stroke, background, missing), 
        ani_icons(factor, stroke, background, missing), 
        pin_icons(factor, stroke, background, missing),
        measure_icons(factor, stroke, background, missing),
        appearance_icons(factor, stroke, background, missing),
        explode_icons(factor, stroke, background, missing),
        axes_icons(factor, stroke, background, missing),
    ]:
        icons.save(theme)